# Project supervised learning - Drunk smurfs

Jean-Baptiste Maene - Denis Topallaj - Lander Pauwels Malengier 

## 0. Data-cleaning

In [1]:
# import dependencies

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-darkgrid')

C:\Users\Jean-Baptiste Maene\AppData\Local\Temp\ipykernel_8448\816365406.py:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


In [2]:
# read the csv file

df = pd.read_csv('train_V2.csv')
df_test = pd.read_csv('score.csv')

## 0.1 Data analyzing

In [3]:
# analyze dataset

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 53 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   income_am              4947 non-null   float64
 1   profit_last_am         4947 non-null   float64
 2   profit_am              4947 non-null   float64
 3   damage_am              4954 non-null   float64
 4   damage_inc             4947 non-null   float64
 5   crd_lim_rec            4947 non-null   float64
 6   credit_use_ic          4947 non-null   float64
 7   gluten_ic              4947 non-null   float64
 8   lactose_ic             4947 non-null   float64
 9   insurance_ic           4947 non-null   float64
 10  spa_ic                 4970 non-null   float64
 11  empl_ic                4999 non-null   float64
 12  cab_requests           4912 non-null   float64
 13  married_cd             5000 non-null   bool   
 14  bar_no                 4947 non-null   float64
 15  spor

In [4]:
df.describe()

,income_am,profit_last_am,profit_am,damage_am,damage_inc,crd_lim_rec,credit_use_ic,gluten_ic,lactose_ic,insurance_ic,...,score2_neg,score3_pos,score3_neg,score4_pos,score4_neg,score5_pos,score5_neg,outcome_profit,outcome_damage_inc,outcome_damage_amount
count,4947.000000,4947.000000,4947.000000,4954.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,...,1.304000e+03,1.261000e+03,1.367000e+03,1.223000e+03,1.324000e+03,1.232000e+03,1493.000000,5000.000000,5000.000000,5000.000000
mean,2281.260158,696.057712,3637.900950,145.952967,0.352335,3298.716394,0.041237,0.024661,0.094199,0.390944,...,4.967340e-01,4.942801e-01,4.985876e-01,4.962065e-01,5.013962e-01,5.009593e-01,5.192953,1967.310930,0.255400,189.970736
std,8365.254507,3051.119275,5726.625669,581.068095,0.889449,4549.646039,0.198858,0.155107,0.292134,0.488011,...,2.897994e-01,2.899165e-01,2.877292e-01,2.886538e-01,2.876226e-01,2.901323e-01,3.159868,1371.061266,0.436129,379.005941
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,-7.871775,10.680000,0.000000,0.000000
25%,229.000000,0.000000,1638.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.454209e-01,2.405574e-01,2.495061e-01,2.474100e-01,2.506703e-01,2.514905e-01,3.124958,1333.320000,0.000000,0.000000
50%,469.000000,52.000000,1889.000000,0.000000,0.000000,1500.000000,0.000000,0.000000,0.000000,0.000000,...,4.985832e-01,4.942465e-01,5.016458e-01,4.933486e-01,5.020603e-01,5.029121e-01,5.188006,1721.235000,0.000000,0.000000
75%,1688.000000,810.000000,3165.500000,0.000000,0.000000,5000.000000,0.000000,0.000000,0.000000,1.000000,...,7.474935e-01,7.449235e-01,7.464826e-01,7.452133e-01,7.493876e-01,7.512817e-01,7.357425,2223.712500,1.000000,202.612500
max,360577.000000,150537.000000,100577.000000,14866.000000,10.000000,30000.000000,1.000000,1.000000,1.000000,1.000000,...,9.986510e-01,9.999999e-01,9.999999e-01,9.999999e-01,9.993125e-01,9.999999e-01,14.776319,31529.000000,1.000000,3157.240000


In [5]:
df.shape

(5000, 53)

#

## 0.2 Drop inconsistent or empty data

### 0.2.1 Dropping reoccurring empty rows

In [98]:
# when these three columns are not filled, the rest of the data is not filled.
# the rows without data are dropped.

indexes = df[(df['income_am'].isnull()) & (df['profit_last_am'].isnull()) & (df['profit_am'].isnull())].index

df = df.drop(index=indexes.array)

print('train', df.shape)

# same with the test file

indexes = df_test[(df_test['income_am'].isnull()) & (df_test['profit_last_am'].isnull()) & (df_test['profit_am'].isnull())].index

df_test = df_test.drop(index=indexes.array)

print('test', df_test.shape)


train (4947, 53)
test (496, 50)


### 0.2.2 Dropping duplicates

In [99]:
df.drop_duplicates()

df_test.drop_duplicates()

,income_am,profit_last_am,profit_am,damage_am,damage_inc,crd_lim_rec,credit_use_ic,gluten_ic,lactose_ic,insurance_ic,...,score1_pos,score1_neg,score2_pos,score2_neg,score3_pos,score3_neg,score4_pos,score4_neg,score5_pos,score5_neg
0,5660.0,4320.0,8640.0,0.0,0.0,8000.0,0.0,0.0,1.0,0.0,...,0.538419,0.396819,0.423742,0.763608,NaN,NaN,NaN,NaN,NaN,NaN
1,3990.0,9.0,3450.0,0.0,0.0,12500.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1158.0,82.0,4194.0,408.0,4.0,12000.0,0.0,0.0,0.0,1.0,...,0.009811,0.592842,NaN,NaN,0.252444,0.724693,0.818064,0.387361,NaN,NaN
3,2451.0,791.0,2119.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,946.0,222.0,2036.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,820.0,216.0,7794.0,1103.0,3.0,9000.0,0.0,0.0,0.0,1.0,...,0.307239,0.660891,NaN,NaN,0.738333,0.914151,NaN,NaN,0.262224,8.060677
496,6092.0,2100.0,3137.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.419981,0.668320,NaN,NaN,NaN,NaN,NaN,NaN
497,2301.0,214.0,2516.0,0.0,0.0,11000.0,0.0,0.0,0.0,0.0,...,0.837325,0.663044,0.697171,0.353229,NaN,NaN,NaN,NaN,NaN,NaN
498,492.0,0.0,3716.0,713.0,2.0,5000.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.461598,4.757132


### 0.2.3 Dropping outliers

In [100]:
# score5_neg has scores ranging from 7995 trillion to -472 trillion and everything in between. score5_neg and score5_pos will be dropped.

# looking at the score of the other four scoring systems we can confidently say that score5_neg can be dropped (score5_pos also has to be dropped, because they both make a pair)

# quantile score can only range 0 < q < 1

columns = ["score5_neg", "score5_pos"]

for col in columns:
    if col in df.columns:
        df.drop(col, axis=1, inplace=True)

# dropping the score5 for the test data set

for col in columns:
    if col in df_test.columns:
        df_test.drop(col, axis=1, inplace=True)

print("train", df.shape)
print("test", df_test.shape)

train (4947, 51)
test (496, 48)


### 0.2.4 Dropping unethical/'useless' columns

In [101]:
# Having gender, place or origin, race ... as a factor to disallow smurfs from entering a hotel is unethical.

if "urban_ic" in df.columns and "neighbor_income" in df.columns and "gender" in df.columns:
	df = df.drop('urban_ic', axis=1)
	df = df.drop('neighbor_income', axis=1)
	df = df.drop('gender', axis=1)
 
if "urban_ic" in df_test.columns and "neighbor_income" in df_test.columns and "gender" in df_test.columns:
	df_test = df_test.drop('urban_ic', axis=1)
	df_test = df_test.drop('neighbor_income', axis=1)
	df_test = df_test.drop('gender', axis=1)

print("train", df.shape)
print("test", df_test.shape)

train (4947, 48)
test (496, 45)


## 0.2 Handle missing data

In [102]:
# These are the columns with missing data

print("train\n", df.isnull().sum()[df.isnull().sum() != 0])

print("\ntest\n", df_test.isnull().sum()[df_test.isnull().sum() != 0])

train
 cab_requests      35
dining_ic         35
presidential      35
tenure_mts       339
tenure_yrs       339
shop_use          35
score1_pos      3722
score1_neg      3633
score2_pos      3738
score2_neg      3643
score3_pos      3686
score3_neg      3580
score4_pos      3724
score4_neg      3623
dtype: int64

test
 cab_requests      1
dining_ic         1
presidential      1
tenure_mts       35
tenure_yrs       35
shop_use          1
score1_pos      374
score1_neg      362
score2_pos      385
score2_neg      372
score3_pos      360
score3_neg      351
score4_pos      377
score4_neg      370
dtype: int64


### 0.2.1 Using mean for missing data

In [103]:
from sklearn.impute import SimpleImputer

df_mean_imputed = df.copy()

# mean worthy columns
columns = ["cab_requests", "dining_ic", "presidential", "shop_use"]

for col in columns:
	mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

	mean_imputer = mean_imputer.fit(np.array(df[col]).reshape(-1, 1))
 
	df_mean_imputed[col] = mean_imputer.transform(np.array(df[col]).reshape(-1, 1))

print("train\n", df_mean_imputed.isnull().sum()[df_mean_imputed.isnull().sum() != 0])


# using mean_imputer for the test data
df_mean_imputed_test = df_test.copy()

# mean worthy columns
columns = ["cab_requests", "dining_ic", "presidential", "shop_use"]

for col in columns:
	mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

	mean_imputer = mean_imputer.fit(np.array(df_test[col]).reshape(-1, 1))
 
	df_mean_imputed_test[col] = mean_imputer.transform(np.array(df_test[col]).reshape(-1, 1))

print("\ntest\n", df_mean_imputed_test.isnull().sum()[df_mean_imputed_test.isnull().sum() != 0])



train
 tenure_mts     339
tenure_yrs     339
score1_pos    3722
score1_neg    3633
score2_pos    3738
score2_neg    3643
score3_pos    3686
score3_neg    3580
score4_pos    3724
score4_neg    3623
dtype: int64

test
 tenure_mts     35
tenure_yrs     35
score1_pos    374
score1_neg    362
score2_pos    385
score2_neg    372
score3_pos    360
score3_neg    351
score4_pos    377
score4_neg    370
dtype: int64


### 0.2.2 Using KNN for missing data

In [104]:
# K-Nearest Neighbors (KNN) Imputation will be used for "tenure_mts" and "tenure_yrs" because it is a good option when the missingness is random, and there is no clear pattern to the missing data.

from sklearn.impute import KNNImputer

df_knn_imputed = df_mean_imputed.copy()

columns = ["tenure_mts", "tenure_yrs"]

for col in columns:
	knn_imputer = KNNImputer(n_neighbors=5)
	imputed_col = knn_imputer.fit_transform(df_knn_imputed[[col]])
	df_knn_imputed[col] = imputed_col

print("train\n", df_knn_imputed.isnull().sum()[df_knn_imputed.isnull().sum() != 0])


# using the KNNImputer for the test data set

df_knn_imputed_test = df_mean_imputed_test.copy()

columns = ["tenure_mts", "tenure_yrs"]

for col in columns:
	knn_imputer = KNNImputer(n_neighbors=5)
	imputed_col = knn_imputer.fit_transform(df_knn_imputed_test[[col]])
	df_knn_imputed_test[col] = imputed_col

print("\ntest\n", df_knn_imputed_test.isnull().sum()[df_knn_imputed_test.isnull().sum() != 0])

train
 score1_pos    3722
score1_neg    3633
score2_pos    3738
score2_neg    3643
score3_pos    3686
score3_neg    3580
score4_pos    3724
score4_neg    3623
dtype: int64

test
 score1_pos    374
score1_neg    362
score2_pos    385
score2_neg    372
score3_pos    360
score3_neg    351
score4_pos    377
score4_neg    370
dtype: int64


### 0.2.3 Handle missing data for scores

In [105]:
# DROPPING THE SCORES BECAUSE CANNOT FIND A GOOD METHOD TO FILL IN THE DATA ATM

columns=["score1_neg", "score1_pos", "score2_pos", "score2_neg", "score3_pos", "score3_neg", "score4_pos", "score4_neg"]

for col in columns:
    if col in df_knn_imputed.columns:
        df_knn_imputed.drop(col, axis=1, inplace=True)

for col in columns:
    if col in df_knn_imputed_test.columns:
        df_knn_imputed_test.drop(col, axis=1, inplace=True)

# THIS IS SO THAT I CAN CONTINUE WITH OTHER STUFF
## IF A GOOD METHOD HAS BEEN FOUND TO FILL IN DATA, THEN THIS BLOCK WILL BE CHANGED 


## 0.3 Changing data types

### 0.3.1 changing floats to int

In [106]:
# These columns are not meant to be floats.
# ex.: You cannot have 0.214 of a child.
# The few that are floats are price or time related.

columns = ["damage_inc", "credit_use_ic", "gluten_ic", "lactose_ic", "insurance_ic", "spa_ic", "empl_ic", "cab_requests", "married_cd", "bar_no", "sport_ic", "age", "marketing_permit", "dining_ic", "presidential", "client_segment", "sect_empl", "prev_stay", "prev_all_in_stay", "divorce", "fam_adult_size", "children_no", "tenure_mts", "tenure_yrs", "company_ic", "claims_no", "nights_booked", "shop_use", "retired", "gold_status"]


for col in columns:
    if not df_knn_imputed[col].empty:
        df_knn_imputed[col] = df_knn_imputed[col].astype(np.uint8)
        
print("train\n", df_knn_imputed.info())

# change type to uint8 for the test data set

for col in columns:
    if not df_knn_imputed_test[col].empty:
        df_knn_imputed_test[col] = df_knn_imputed_test[col].astype(np.uint8)

print("\ntest\n", df_knn_imputed_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4947 entries, 0 to 4999
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   income_am              4947 non-null   float64
 1   profit_last_am         4947 non-null   float64
 2   profit_am              4947 non-null   float64
 3   damage_am              4947 non-null   float64
 4   damage_inc             4947 non-null   uint8  
 5   crd_lim_rec            4947 non-null   float64
 6   credit_use_ic          4947 non-null   uint8  
 7   gluten_ic              4947 non-null   uint8  
 8   lactose_ic             4947 non-null   uint8  
 9   insurance_ic           4947 non-null   uint8  
 10  spa_ic                 4947 non-null   uint8  
 11  empl_ic                4947 non-null   uint8  
 12  cab_requests           4947 non-null   uint8  
 13  married_cd             4947 non-null   uint8  
 14  bar_no                 4947 non-null   uint8  
 15  spor

## 0.4 Storing cleaned dataset

In [107]:
# the cleaned csv will now be in the file train_V2_cleaned and will be used for the rest of the calculations

df_knn_imputed.to_csv('train_V2_cleaned.csv', index=False)
df_knn_imputed.to_csv('score_cleaned.csv', index=False)